# https://github.com/redtreeai/easy_text_emotion

In [3]:
# -*- coding: utf-8 -*-
# @File    :  emotion_eng.py
# @Desc : 文本情感分析算法 （英文）

class AllList():  # 存储所有列表信息的对象
    positive_words_eng = [] #正螚量词
    negative_words_eng = [] #负能量词
    level1_words_eng = [] #程度1
    level2_words_eng = [] #程度2
    level3_words_eng = [] #程度3
    level4_words_eng = [] #程度4
    level5_words_eng = [] #程度5
    level6_words_eng = [] #程度6
    fouding_words_eng = [] #否定词
    pass

def getAllList():  # 提取所有规则列表（后期要改为多线程提取）

    allList = AllList()

    # 情感分析(英文)
    file = open(r"D:\1\pos.txt", encoding='UTF-8')#这些文件都要的！！！
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.positive_words_eng.append(checkTr)

    file = open(r"D:\1\neg.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.negative_words_eng.append(checkTr)

    file = open(r"D:\1\level1.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level1_words_eng.append(checkTr)

    file = open(r"D:\1\level2.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level2_words_eng.append(checkTr)

    file = open(r"D:\1\level3.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level3_words_eng.append(checkTr)

    file = open(r"D:\1\level4.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level4_words_eng.append(checkTr)

    file = open(r"D:\1\level5.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level5_words_eng.append(checkTr)

    file = open(r"D:\1\level6.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.level6_words_eng.append(checkTr)

    file = open(r"D:\1\fouding.txt", encoding='UTF-8')
    while 1:
        line = file.readline()
        if not line:
            break
        pass
        checkTr = str(line).replace('\n', '')
        allList.fouding_words_eng.append(checkTr)

    return allList

In [4]:
import nltk
import random
#https://github.com/redtreeai/easy_text_emotion

In [5]:
all_list = getAllList()


In [6]:
def cutSentence(input):  
    res = nltk.word_tokenize(input)  # 默认是精确模式
    return res

In [7]:
def reduceFunc(reduceTimes):  #适用于累加条件下的衰减法 （pos and neg）
    addY=(10/reduceTimes)/10
    return addY #衰减后当此追加的情感值

In [8]:
def levelReduceFunc(levelReduceTimes,level,type): #适用于区间叠乘条件下的衰减法 （level）
    if level==6:
        levela=2  #区间下限
        levelb=2.5 #区间上限
        levelER=0.67 #衰减系数

    if level==5:
        levela=1.6  #区间下限
        levelb=1.9 #区间上限
        levelER=0.67 #衰减系数

    if level==4:
        levela=1.2  #区间下限
        levelb=1.6 #区间上限
        levelER=0.67 #衰减系数

    if level==3:
        levela=0.6  #区间下限
        levelb=0.9 #区间上限
        levelER=1.23 #衰减系数

    if level == 2:
        levela = 0.4  # 区间下限
        levelb = 0.7  # 区间上限
        levelER = 1.23  # 衰减系数

    if level == 1:
        levela = 0.2  # 区间下限
        levelb = 0.5  # 区间上限
        levelER = 1.23  # 衰减系数

    if level >= 4:

       if type == 'topLimit':   #上限衰减
            levelbCheck = levelb * (levelER ** (levelReduceTimes - 1))
            if levelbCheck<=1.001:
                levelbCheck = 1.001
            return levelbCheck
       if type == 'lowerLimit': #下限衰减
            levelaCheck = levela * (levelER ** (levelReduceTimes - 1))
            if levelaCheck<=1.000:
                levelaCheck = 1.000
            return  levelaCheck

    if level <=3 :

        if type == 'topLimit':  # 上限衰减
            levelbCheck = levelb * (levelER ** (levelReduceTimes - 1))
            if levelbCheck >= 1.000:
                levelbCheck = 1.000
            return levelbCheck
        if type == 'lowerLimit':  # 下限衰减
            levelaCheck = levela * (levelER ** (levelReduceTimes - 1))
            if levelaCheck >= 0.998:
                levelaCheck = 0.998
            return levelaCheck



In [9]:

def checkMoodValue(segWord): #获取句子的情感能量值 mv =(posV*(functionE)+negV*(functionE))*isFouDing*chekLevel
    MoodValue = 0                  #functionE 为待加算法，用于处理重复数据的量级衰减
    isFouDing = 1
    checkLevel=1
    posreduceTimes=1
    negreduceTimes=1
    level6ReduceTimes=1
    level5ReduceTimes=1
    level4ReduceTimes=1
    level3ReduceTimes = 1
    level2ReduceTimes = 1
    level1ReduceTimes = 1
    # 正能量词Check
    for onePos in all_list.positive_words_eng:
        if str(onePos).__contains__(' '):
            if str(onePos) in str(segWord):
                MoodValue = MoodValue + reduceFunc(posreduceTimes);
                posreduceTimes = posreduceTimes + 1;
                break
        else:
            if str(onePos) in cutSentence(str(segWord)):
                MoodValue = MoodValue + reduceFunc(posreduceTimes);
                posreduceTimes = posreduceTimes + 1;
                break

    # 负能量词Check
    for oneNeg in all_list.negative_words_eng:
        if str(oneNeg).__contains__(' '):
            if str(oneNeg) in str(segWord):
                MoodValue = MoodValue - reduceFunc(negreduceTimes);
                negreduceTimes = negreduceTimes + 1;
                break
        else:
            if str(oneNeg) in cutSentence(str(segWord)):
                MoodValue = MoodValue - reduceFunc(negreduceTimes);
                negreduceTimes = negreduceTimes + 1;
                break
    # 否定词Check
    for fdword in all_list.fouding_words_eng:
        if str(fdword).__contains__(' '):
            if str(fdword) in str(segWord):
                isFouDing = isFouDing * (-1)
                break
        else:
            if str(fdword) in cutSentence(str(segWord)):
                isFouDing = isFouDing * (-1)
                break
    # 程度级Check 矫正系数 er
    # level1 矫正系数(0.2~0.5)
    # level2 矫正系数(0.4~0.7)
    # level3 矫正系数(0.6~0.9)
    # level4 矫正系数(1.2~1.6)
    # level5 矫正系数(1.6~1.9)
    # level6 矫正系数(2.0~2.5)

    for oneLevel in all_list.level1_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level1ReduceTimes, 1, 'lowerLimit'),
                                    levelReduceFunc(level1ReduceTimes, 1, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level1ReduceTimes, 1, 'lowerLimit'),
                                    levelReduceFunc(level1ReduceTimes, 1, 'topLimit'))
                checkLevel = checkLevel * er
                break
    for oneLevel in all_list.level2_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level2ReduceTimes, 2, 'lowerLimit'),
                                    levelReduceFunc(level2ReduceTimes, 2, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level2ReduceTimes, 2, 'lowerLimit'),
                                    levelReduceFunc(level2ReduceTimes, 2, 'topLimit'))
                checkLevel = checkLevel * er
                break
    for oneLevel in all_list.level3_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level3ReduceTimes, 3, 'lowerLimit'),
                                    levelReduceFunc(level3ReduceTimes, 3, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level3ReduceTimes, 3, 'lowerLimit'),
                                    levelReduceFunc(level3ReduceTimes, 3, 'topLimit'))
                checkLevel = checkLevel * er
                break
    for oneLevel in all_list.level4_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level4ReduceTimes, 4, 'lowerLimit'),
                                    levelReduceFunc(level4ReduceTimes, 4, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level4ReduceTimes, 4, 'lowerLimit'),
                                    levelReduceFunc(level4ReduceTimes, 4, 'topLimit'))
                checkLevel = checkLevel * er
                break
    for oneLevel in all_list.level5_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level5ReduceTimes, 5, 'lowerLimit'),
                                    levelReduceFunc(level5ReduceTimes, 5, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level5ReduceTimes, 5, 'lowerLimit'),
                                    levelReduceFunc(level5ReduceTimes, 5, 'topLimit'))
                checkLevel = checkLevel * er
                break
    for oneLevel in all_list.level6_words_eng:
        if str(oneLevel).__contains__(' '):
            if str(oneLevel) in str(segWord):
                er = random.uniform(levelReduceFunc(level6ReduceTimes, 6, 'lowerLimit'),
                                    levelReduceFunc(level6ReduceTimes, 6, 'topLimit'))
                checkLevel = checkLevel * er
                break
        else:
            if str(oneLevel) in cutSentence(str(segWord)):
                er = random.uniform(levelReduceFunc(level6ReduceTimes, 6, 'lowerLimit'),
                                    levelReduceFunc(level6ReduceTimes, 6, 'topLimit'))
                checkLevel = checkLevel * er
                break

    MoodValue = MoodValue * isFouDing * checkLevel
    return MoodValue



In [10]:
def getMoodValue(text):
    try:
        text = str(text).replace('，', '，|')
        text = str(text).replace('。', '。|')
        text = str(text).replace(',', '，|')
        text = str(text).replace('.', '。|')
        text = str(text).replace('!', '!|')
        text = str(text).replace('！', '！|')
        text = str(text).replace('？', '？|')
        text = str(text).replace('?', '?|')
        tsp_list = text.split('|')
        all_mv = 0
        re_obj = []

        for tl in tsp_list:
            if not (tl == '' or tl == ' ' ):
                tmp_MoodValue = checkMoodValue(tl)
                all_mv = all_mv + tmp_MoodValue
                cobj = {'text':tl,'value':round(tmp_MoodValue,6)}
                re_obj.append(cobj)

        res = {'all_value':round(all_mv,6),'split':re_obj}
        return res
    except Exception as err:
        print('文本情感分析失败'+str(err))
        return 'error'

In [48]:
def getMoodValue2(text):
    try:
        text = str(text).replace('，', '，|')
        text = str(text).replace('。', '。|')
        text = str(text).replace(',', '，|')
        text = str(text).replace('.', '。|')
        text = str(text).replace('!', '!|')
        text = str(text).replace('！', '！|')
        text = str(text).replace('？', '？|')
        text = str(text).replace('?', '?|')
        tsp_list = text.split('|')
        all_mv = 0
        re_obj = []

        for tl in tsp_list:
            if not (tl == '' or tl == ' ' ):
                tmp_MoodValue = checkMoodValue(tl)
                all_mv = all_mv + tmp_MoodValue

        res = {'all_value':round(all_mv,6)}
        return res
    except Exception as err:
        print('文本情感分析失败'+str(err))
        return 'error'

In [ ]:
# %load main.py
# @Time    : 19-3-28 上午10:56
# @Author  : Redtree
# @File    : main.py
# @Desc :
if __name__ == '__main__':
    while 1>0 :
        lg = input("switch language,type 'CN' or 'ENG' ,if you want to exit ,type '0' "+'\n')
        if lg == 'CN':
            print("type chinese text to get result,if you want to exit ,type '0' ")
            while 1 > 0:
                input_text = input('\n')
                out_put = emotion_cn.getMoodValue2(input_text)
                print(out_put)
                if input_text == '0':
                    break
        elif lg == 'ENG':
            print("type english text to get result,if you want to exit ,type '0' ")
            while 1 > 0:
                input_text = input('\n')
                out_put = getMoodValue2(input_text)
                print(out_put)
                if input_text == '0':
                    break
        elif lg =='0':
            break
#先选ENG，然后copy评论到输入框里，自动会有结果

switch language,type 'CN' or 'ENG' ,if you want to exit ,type '0' 
ENG
type english text to get result,if you want to exit ,type '0' 

ENG
{'all_value': 0}

saddddd
{'all_value': 0}

sad
{'all_value': -1.0}

The easiest subject EVER EVER!!! I I only studied biology at f.3, but I could still catch up the lessons easily. Highly recommend this subject to all of you guys
{'all_value': 1.0}

horrible tutor, I have ever met.
{'all_value': -1.0}

Don’t take this course if you want a grade above a B.
{'all_value': 0.241265}

If you want to get A or A+, it is very hard. Helena will take all your scores in this course into consideration. This summer, you may fail to get an A if your score of final is less than 80, even your total score is high enough. You will also fail to get an A+ if your score of final is only a bit higher than 80 while your total score is even higher than 90, which means your score may be even higher than some students who get an A+. That is to say, Helena prefers the final.

{'all_value': 2.357041}


{'all_value': 0}

4 lab reportsfew simple quizzes in lesson one group project (4~5 ppl per grp)exam only have one pages mostly related to lab reportmy whole group with or without science background also get good result
{'all_value': 1.0}

People who love philosophy would find this course interesting as it covers topics like justice,governance,moral values etc.The teacher is willing to answer your questions during lessons and after lessons.However,the guidelines for online test and individual paper is a bit unclear so you can't tell whether you have written in a appropriate way and have met his requirements.
{'all_value': 2.0}

I took the course during COVID-19 so the class was done online. However, Alex Wong chose not to record and upload the lectures online. When students emailed him saying they’re residing in countries where the time zones are very different he responded rudely and did not care.I would say approximately 20% of the class is taught in Chinese 

{'all_value': -1.0}

Difficult course if you're not a STEM student, but for STEM students it's fun. Compulsory for EE. You'll have to work hard and lab assistants can and cannot be helpful depending on their mood.
{'all_value': 0.0}

Took it 2019/20 Sem ASumie CHAN sucks. My work was much above average, if not the best, in class, and she herself said that in her comments, yet gave me an average grade. The people in my class who could barely speak english (and never bothered to participate (there are marks for that)) got a better grade than me. I so regret I didn't pass fail is. There is no pleasing her.
{'all_value': 0.1134}

2020 SEM BPre-proposal 1 Assignment MidtermFinal Report (Business Plan)Final PresentationFinal Exam The course is easy and the workload is okay. Special knowledge is not required. The midterm was super uncomfortable: a lot of questions with the numbers, in the beginning, so you have to scroll every time. Because of the online semester, there were time limits and h

{'all_value': 1.322771}

Weith's requirements for the essay are very unique, only one source for one paragraph and the structure of the essay is fixed to be 5 paragraphs. No flexibility at all. Am I a primary student?? Really??p.s. (Don't expect that he will upload the results of your work on canvas. )
{'all_value': 1.0}

easy course materials, If u know a few programming, that's enough for u to get a grade above b. Although there are quite many stuffs to do, many of them is not important, just ignore it is okay.
{'all_value': 2.0}

If you want to be respected by others, DO NOT choose CALVIN WONG .He is the one who will SATIRIZE you when you just ask a simply question about a topic in the course and he says "this is a common sense" but never give you constructive response.
{'all_value': 1.0}

I am from College of business, winner of International Olympics of Physics and I got the A-, don't know what is teacher's grading standard.
{'all_value': 0}

Nice course if you like philosophical 